In [ ]:
## We need city csv; we need places shp file for the FIPS

In [18]:
import pandas as pd
import geopandas as gpd
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

In [19]:
# load city info used in Heat Pump Calculator (HPC) 
# from Alan's original data repository
# load pickle file to preserve the data format 
city_url = 'https://github.com/alanmitchell/akwlib-export/blob/main/data/v01/city.pkl' + '?raw=true'
city_HPC = pd.read_pickle(city_url, compression = 'bz2')

In [20]:
city_HPC.head()

,Name,Latitude,Longitude,ERHRegionID,WAPRegionID,ImprovementCostLevel,FuelRefer,FuelCityID,Oil1Price,Oil2Price,...,TMYname,ElecUtilities,GasPrice,aris_city,census_city,census_area,ancsa_region,railbelt,hub,avg_elec_usage
ID,,,,,,,,,,,,,,,,,,,,,
1,Anchorage,61.151960,-149.864100,2,2,1,0,NaN,4.45,NaN,...,"ANCHORAGE INTL AP, AK","[(Chugach Electric South- Residential, 1), (Ch...",0.988132,Anchorage,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,True,"[674.3176110825, 616.54369648225, 568.80201815..."
2,Adak,51.848896,-176.635422,2,3,5,0,NaN,7.30,6.64,...,"ADAK NAS, AK","[(Adak -TDX Residential, 677), (Adak - TDX Com...",NaN,Adak,Adak city,Aleutians West Census Area,Aleut,Affordable Energy Strategy Area,False,"[471.936998147, 423.45531781, 460.81549341, 42..."
3,Akutan,54.134945,-165.772446,2,3,5,0,NaN,4.93,NaN,...,"DUTCH HARBOR, AK","[(Akutan, City of - Residential, 9), (Akutan, ...",NaN,Akutan,Akutan city,Aleutians East Borough,Aleut,Affordable Energy Strategy Area,False,"[500.99156233, 439.984315846, 456.510655517, 4..."
4,Allakaket,66.561386,-152.649445,4,4,5,0,NaN,7.00,8.00,...,"BETTLES FIELD, AK","[(Allakaket - APT - Residential, 249), (Allaka...",NaN,Allakaket,Allakaket city,Yukon-Koyukuk Census Area,Doyon,Affordable Energy Strategy Area,False,"[338.509000744, 291.889556239, 276.804469551, ..."
5,Angoon,57.496647,-134.578900,1,1,3,0,NaN,6.20,NaN,...,"SITKA JAPONSKI AP, AK","[(Inside Passage Electric Coop - Residential, ...",NaN,Angoon,Angoon city,Hoonah-Angoon Census Area,Sealaska Corporation,Affordable Energy Strategy Area,False,"[277.080558981, 257.757954545, 237.15561277, 2..."


In [21]:
# download and unzip blocks manually because of its big size
zipurl = 'https://live.laborstats.alaska.gov/cen/maps/gis/akplaces_2010.zip'
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('/tmp/Places2010')

places = gpd.read_file('/tmp/Places2010/')

In [22]:
# match city with places (FIPS)
city_HPC['name2match'] = city_HPC['census_city'].str.lower()
places['name2match'] = places['NAME'].str.lower()

In [23]:
# check if matching is perfect
city_HPC['name2match'].isin(places['name2match']).mean()

1.0

In [24]:
places2match = places[['name2match', 'FIPS']]

In [25]:
city_HPC.reset_index().merge(places2match, on='name2match')

,ID,Name,Latitude,Longitude,ERHRegionID,WAPRegionID,ImprovementCostLevel,FuelRefer,FuelCityID,Oil1Price,...,GasPrice,aris_city,census_city,census_area,ancsa_region,railbelt,hub,avg_elec_usage,name2match,FIPS
0,1,Anchorage,61.151960,-149.864100,2,2,1,0,NaN,4.450000,...,0.988132,Anchorage,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,True,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
1,275,Girdwood,60.941910,-149.168564,2,2,1,0,1.0,4.500000,...,0.988132,Girdwood,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
2,298,Indian,60.990900,-149.504211,2,2,1,0,NaN,4.450000,...,0.988132,Indian,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
3,299,Chugiak,61.388344,-149.484146,2,2,1,0,NaN,4.450000,...,0.988132,Chugiak,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
4,300,Eagle River,61.320835,-149.569962,2,2,1,0,1.0,4.450000,...,0.988132,Eagle River,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,304,North Pole,64.749275,-147.348969,4,4,2,0,293.0,4.590000,...,NaN,North Pole,North Pole city,Fairbanks North Star Borough,Doyon,Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",north pole city,0255910
273,307,Salcha,64.546920,-147.006409,4,4,3,0,NaN,5.090000,...,NaN,Salcha,Salcha CDP,Fairbanks North Star Borough,Doyon,Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",salcha cdp,0266550
274,309,Ester,64.853310,-147.979614,4,4,2,0,293.0,4.590000,...,NaN,Ester,Ester CDP,Fairbanks North Star Borough,Doyon,Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",ester cdp,0223460
275,311,Whale Pass,56.115063,-133.154816,1,1,4,0,0.0,5.580000,...,NaN,Whale Pass,Whale Pass CDP,Prince of Wales-Hyder Census Area,Sealaska Corporation,Affordable Energy Strategy Area,False,"[511.306363975, 477.809378262, 423.544981799, ...",whale pass cdp,0284000


In [26]:

merged_data = city_HPC.merge(places2match, on='name2match')
merged_data

,Name,Latitude,Longitude,ERHRegionID,WAPRegionID,ImprovementCostLevel,FuelRefer,FuelCityID,Oil1Price,Oil2Price,...,GasPrice,aris_city,census_city,census_area,ancsa_region,railbelt,hub,avg_elec_usage,name2match,FIPS
0,Anchorage,61.151960,-149.864100,2,2,1,0,NaN,4.450000,NaN,...,0.988132,Anchorage,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,True,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
1,Girdwood,60.941910,-149.168564,2,2,1,0,1.0,4.500000,NaN,...,0.988132,Girdwood,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
2,Indian,60.990900,-149.504211,2,2,1,0,NaN,4.450000,NaN,...,0.988132,Indian,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
3,Chugiak,61.388344,-149.484146,2,2,1,0,NaN,4.450000,NaN,...,0.988132,Chugiak,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
4,Eagle River,61.320835,-149.569962,2,2,1,0,1.0,4.450000,NaN,...,0.988132,Eagle River,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,North Pole,64.749275,-147.348969,4,4,2,0,293.0,4.590000,NaN,...,NaN,North Pole,North Pole city,Fairbanks North Star Borough,Doyon,Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",north pole city,0255910
273,Salcha,64.546920,-147.006409,4,4,3,0,NaN,5.090000,NaN,...,NaN,Salcha,Salcha CDP,Fairbanks North Star Borough,Doyon,Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",salcha cdp,0266550
274,Ester,64.853310,-147.979614,4,4,2,0,293.0,4.590000,NaN,...,NaN,Ester,Ester CDP,Fairbanks North Star Borough,Doyon,Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",ester cdp,0223460
275,Whale Pass,56.115063,-133.154816,1,1,4,0,0.0,5.580000,5.57,...,NaN,Whale Pass,Whale Pass CDP,Prince of Wales-Hyder Census Area,Sealaska Corporation,Affordable Energy Strategy Area,False,"[511.306363975, 477.809378262, 423.544981799, ...",whale pass cdp,0284000


In [27]:
merged_data['FIPS'] = pd.to_numeric(merged_data['FIPS'])

In [28]:
merged_data.columns.tolist()

['Name',
 'Latitude',
 'Longitude',
 'ERHRegionID',
 'WAPRegionID',
 'ImprovementCostLevel',
 'FuelRefer',
 'FuelCityID',
 'Oil1Price',
 'Oil2Price',
 'PropanePrice',
 'BirchPrice',
 'SprucePrice',
 'CoalPrice',
 'SteamPrice',
 'HotWaterPrice',
 'MunicipalSalesTax',
 'BoroughSalesTax',
 'TMYid',
 'TMYname',
 'ElecUtilities',
 'GasPrice',
 'aris_city',
 'census_city',
 'census_area',
 'ancsa_region',
 'railbelt',
 'hub',
 'avg_elec_usage',
 'name2match',
 'FIPS']

In [29]:
cols = [
 'Name',
 'FIPS',
 'TMYid',
 'Latitude',
 'Longitude',
 'ERHRegionID',
 'WAPRegionID',
 'ImprovementCostLevel',
 'FuelRefer',
 'FuelCityID',
 'Oil1Price',
 'Oil2Price',
 'PropanePrice',
 'BirchPrice',
 'SprucePrice',
 'CoalPrice',
 'SteamPrice',
 'HotWaterPrice',
 'MunicipalSalesTax',
 'BoroughSalesTax',
 'TMYname',
 'ElecUtilities',
 'GasPrice',
 'aris_city',
 'census_city',
 'census_area',
 'ancsa_region',
 'railbelt',
 'hub',
 'avg_elec_usage',
 'name2match'
 ]

In [30]:
# # Rearrange the columns
merged_data = merged_data[cols]

In [31]:
# start with here because the data is already merged with FIPS
merged_data.head()

,Name,FIPS,TMYid,Latitude,Longitude,ERHRegionID,WAPRegionID,ImprovementCostLevel,FuelRefer,FuelCityID,...,ElecUtilities,GasPrice,aris_city,census_city,census_area,ancsa_region,railbelt,hub,avg_elec_usage,name2match
0,Anchorage,203000,702730,61.151960,-149.864100,2,2,1,0,NaN,...,"[(Chugach Electric South- Residential, 1), (Ch...",0.988132,Anchorage,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,True,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality
1,Girdwood,203000,702757,60.941910,-149.168564,2,2,1,0,1.0,...,"[(Chugach Electric South- Residential, 1), (Ch...",0.988132,Girdwood,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality
2,Indian,203000,702730,60.990900,-149.504211,2,2,1,0,NaN,...,"[(Chugach Electric South- Residential, 1), (Ch...",0.988132,Indian,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality
3,Chugiak,203000,702746,61.388344,-149.484146,2,2,1,0,NaN,...,"[(Matanuska Electric Assn. - Residential, 77),...",0.988132,Chugiak,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality
4,Eagle River,203000,702746,61.320835,-149.569962,2,2,1,0,1.0,...,"[(Matanuska Electric Assn. - Residential, 77),...",0.988132,Eagle River,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,False,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality


In [35]:
merged_data.to_pickle('data/v01/city.pkl', compression = "bz2")
merged_data.to_csv('data/v01/city.csv', index=False)

### Create a new meta file

In [36]:
# write a new meta data file
merged_data.shape[0] # 277 cities 

277

In [37]:
ERA5_meta = merged_data[['name2match', 'FIPS', 'TMYid']]
ERA5_meta = ERA5_meta.rename(columns={'TMYid':'tmy_id', 'name2match':'city'})
ERA5_meta.head()

,city,FIPS,tmy_id
0,anchorage municipality,203000,702730
1,anchorage municipality,203000,702757
2,anchorage municipality,203000,702730
3,anchorage municipality,203000,702746
4,anchorage municipality,203000,702746


In [39]:
tym3_meta = pd.read_csv("Data/v01/tmy3/tmy3_meta.csv")
tym3_meta.head()

,tmy_id,city,state,utc_offset,latitude,longitude,elevation,db_temp_avg,rh_avg,wind_spd_avg,heating_design_temp
0,700197,SELAWIK,AK,-9.0,66.600,-160.000,26.24672,24.855438,78.158676,9.655307,-36.6
1,700260,BARROW W POST-W ROGERS ARPT [NSA - ARM],AK,-9.0,71.320,-156.620,32.80840,10.729712,84.193950,12.178734,-34.3
2,700637,DEADHORSE,AK,-9.0,70.200,-148.483,75.45932,14.890158,81.881279,11.842146,-41.3
3,701043,POINT HOPE (AWOS),AK,-9.0,68.350,-166.800,13.12336,24.074966,77.388927,15.689121,-24.1
4,701195,SHISHMAREF (AWOS),AK,-9.0,66.267,-166.050,6.56168,25.309137,78.231279,13.223661,-26.2


In [40]:
tym3_meta2 = tym3_meta[['tmy_id', 'heating_design_temp']]

In [41]:
ERA5_meta2 = ERA5_meta.merge(tym3_meta2, on='tmy_id',how='left')

In [42]:
ERA5_meta2 = ERA5_meta2.set_index('FIPS')

In [43]:
ERA5_meta2.head()

,city,tmy_id,heating_design_temp
FIPS,,,
203000,anchorage municipality,702730,-4.5
203000,anchorage municipality,702757,9.8
203000,anchorage municipality,702730,-4.5
203000,anchorage municipality,702746,-8.5
203000,anchorage municipality,702746,-8.5


In [ ]:
ERA5_meta2.to_csv('data/v01/ERA5/ERA5_meta.csv', index=False)
ERA5_meta2.to_pickle('data/v01/ERA5/ERA5_meta.pkl', compression = "bz2")